To fix list:
- The 'year' scraping part still doesn't work

For future reference:
2001 to 2006 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "FF", "FA"]

2007 to 2009 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "FF", "FA", "AF", "SC"]

2010 to 2022 has format:
columns=["Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "CL", "CG", "R50", "FF", "FA", "AF", "SC"]

Note that 2010 onwards also has advanced stats available: add afl/footy/ft_match_statistics?mid=5089&advv=Y

In [56]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from tqdm import tqdm

first_game = 1
last_game = 10741

dfs = []
bad_match_ids = []
for match_id in (range(first_game, last_game+1)):
    try:
        print(match_id)
        url = f"https://www.footywire.com/afl/footy/ft_match_statistics?mid={match_id}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the table element with the match statistics
        match_stats_table = soup.find("table", {"id": "match-statistics-div"})
        if match_stats_table is None:
            print(f"No match statistics div found for match ID {match_id}")
            continue

        # Find the round number and year
        if match_id < 1840:
            year_round = soup.find('td', {"class": "lnorm", "height":"22"})
            round = year_round.text[0:year_round.text.find(',')]
            year = year_round.text[-4:]
        else:
            year_round = soup.find('td', {"class": "lnorm", "height":"22"})
            round = year_round.text[0:year_round.text.find(',')]
            year_round = soup.find('td', {"class": "lnorm", "height":"22"}).find_next('td', {"class": "lnorm", "height":"22"})
            year = year_round.text[-4:]

        if 'final' in round.lower():
            print(f"There is 'final' in this round: {round}")
            continue

        # Find the rows with player statistics
        player_stats_rows = match_stats_table.find_all("tr", {"class": ["darkcolor", "lightcolor"], "onmouseover": "this.className='highlightcolor';", "onmouseout": ["this.className='darkcolor';", "this.className='lightcolor';"]})
        first_team = soup.find("td", {"class": "innertbtitle", "align": "left"}).find("b").text
        first_team = first_team[:first_team.find("Match Statistics")].strip()
        second_team = soup.find("td", {"class": "innertbtitle", "align": "left"}).find_next("td", {"class": "innertbtitle", "align": "left"}).find("b").text
        second_team = second_team[:second_team.find("Match Statistics")].strip()

        if 'defeats' in soup.title.text.lower():
            winner = first_team
        else:
            winner = second_team

        # Extract the data from each row and create a pandas dataframe
        data = []
        for i, row in enumerate(player_stats_rows):
            team = first_team if i < len(player_stats_rows)/2 else second_team
            cols = row.find_all("td")
            cols = [col.text.strip() for col in cols]
            if len(cols) == 11:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "FF", "FA"]
            elif len(cols) == 15:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "FF", "FA", "AF", "SC"]
            elif len(cols) == 18:
                result = 1 if team == winner else 0
                new_row = [year, round, team, result] + cols
                data.append(new_row)
                columns=["Year", "Round", "Team Name", "Win", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "GA", "I50", "CL", "CG", "R50", "FF", "FA", "AF", "SC"]

        if data:
            df = pd.DataFrame(data, columns=columns)
            df["Match ID"] = match_id   

            # Get Brownlow votes
            brownlow_votes = soup.find(string=re.compile("Brownlow Votes:"))
            if brownlow_votes:
                votes = {}
                for i in range(3):
                    player = brownlow_votes.find_next("a").text.strip()
                    player_parts = player.split(" ")
                    last_name = player_parts[-1]
                    first_initial = player_parts[0][0]
                    player_reversed = " ".join(player_parts[::-1])
                    votes[player_reversed] = 3-i
                    brownlow_votes = brownlow_votes.find_next("a")
                df["Votes"] = df["Player"].apply(lambda x: votes.get(x.split()[-1] + ' ' + x.split()[0][0], 0))
            else:
                df["Votes"] = 0

            dfs.append(df)
            print(f"Processed match ID {match_id}")
    except Exception as e:
        print(f"Error processing match ID {match_id}: {e}")
        bad_match_ids.append(match_id)

# Concatenate all dataframes into a single one
df_final = pd.concat(dfs)

# Save the data to a CSV file
df_final.to_csv("player_stats.csv", index=False)


1
Processed match ID 1
2
Processed match ID 2
3
Processed match ID 3
4
Processed match ID 4
5
Processed match ID 5
6
Processed match ID 6
7
Processed match ID 7
8
Processed match ID 8
9
Processed match ID 9
10
Processed match ID 10
11
Processed match ID 11
12


In [4]:
import pandas as pd

with open('player_stats.csv', 'r') as f:
    lines = f.readlines()
    lines = [line.replace('\'', '') for line in lines]

# sum votes, group by match_id
df = pd.read_csv('player_stats.csv')
df = df.groupby(['Match ID']).sum().reset_index()
df = df[['Match ID', 'Votes']]
# get all match_ids with votes != 6
df = df[df['Votes'] != 6]
df

C:\Users\tobyp\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Match ID,Votes
21,22,3
30,31,4
57,58,5
200,210,7
223,233,8
...,...,...
6358,10704,5
6360,10706,3
6368,10714,7
6378,10724,4
